BÀI THỰC HÀNH MÔ HÌNH NGÔN NGỮ

Nội dung:

1) Mô hình ngôn ngữ N-GRAM:

- Huấn luyện mô hình ngôn ngữ với ngữ liệu ham và spam.

- Dùng mô hình để tạo sinh các câu ham và spam.

- Dùng mô hình phân lớp được huấn luyện trên ngữ liệu ham và spam để đánh giá trên tập câu đã được tạo sinh.

2) Mô hình ngôn ngữ với RNN (nội dung tương tự, cần sử dụng GPU T4 để tăng tốc độ huấn luyện):
- Huấn luyện mô hình ngôn ngữ với ngữ liệu ham và spam.

- Dùng mô hình để tạo sinh các câu ham và spam.

- Dùng mô hình phân lớp được huấn luyện trên ngữ liệu ham và spam để đánh giá trên tập câu đã được tạo sinh.

In [ ]:
!pip install --upgrade kagglehub
!pip install --upgrade scikit-learn
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 805.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

CHUẨN BỊ NGỮ LIỆU

Chúng ta sẽ dùng ngữ liệu sms-spam

In [ ]:
import torch
import random
from torch import nn
import numpy as np
import pandas as pd
import random
import kagglehub
import nltk
from nltk import bigrams, trigrams
from collections import defaultdict
import os


In [ ]:
nltk.download('punkt_tab')
path = kagglehub.dataset_download("uciml/sms-spam-collection-dataset")
print("!cp {}/spam.csv /content/".format(path))
os.system("cp {}/spam.csv /content/".format(path))

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


!cp /kaggle/input/sms-spam-collection-dataset/spam.csv /content/


0

In [ ]:
dataset = pd.read_csv("spam.csv", encoding="windows-1252")
dataset.info()
dataset = dataset.drop(dataset[["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"]], axis=1)
dataset.rename(columns = {"v1":"Class", "v2":"Message"}, inplace = True)
dataset.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


,Class,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


I) Mô hình ngôn ngữ N-gram

Trong ví dụ này, chúng ta dùng mô hình ngôn ngữ Bi-gram.

Chúng ta cài đặt hàm tạo mô hình ngôn ngữ Bi-gram và hàm tạo sinh câu từ mô hình ngôn ngữ Bi-gram.

In [ ]:
def buildBigramModel(t, vocab, alpha):
  size = len(vocab)
  model = np.zeros([size, size])
  for sent in t:
    grams = ['<START>'] + nltk.word_tokenize(sent) + ['<END>']
    bg = list(bigrams(grams))
    for w1, w2 in bg:
      model[vocab[w1]][vocab[w2]] += 1

  for i in range(size):
    N = sum(model[i]) + alpha * size
    for j in range(size):
      model[i][j] = (model[i][j] + alpha) / N

  return model

def generate(model, vocab, dictionary, word, step, nstep):
  if step == nstep:
    return "<END>"
  d = model[vocab[word]]
  size = len(model)
  c = 0
  ws = []
  ps = []
  for i in range(size):
    c += d[i]
    ps.append(c)
  if c == 0:
    return "<END>"

  p = random.uniform(0,1)
  for i in range(len(ps)):
    if ps[i] >= p:
      break
  nextWord = ""
  if i == range(len(ps)):
    nextWord = dictionary[-1]
  else:
    nextWord = dictionary[i]
  if nextWord == "<END>":
    return nextWord
  return "{} {}".format(nextWord, generate(model, VOCAB, dictionary, nextWord, step+1, nstep ))



Tạo từ điển và từ vựng để tạo one-hot vector và xác định từ dựa vào one-hot vector

In [ ]:
msgs = list(dataset["Message"])
clss = list(dataset["Class"])
hamtext = []
spamtext = []
VOCAB = {}
n = 0
for i in range(len(msgs)):
  words = nltk.word_tokenize(msgs[i])
  for w in words:
    if VOCAB.get(w) == None:
      VOCAB[w] = n
      n += 1

  if clss[i] == "ham":
    hamtext.append(msgs[i])
  else:
    spamtext.append(msgs[i])

VOCAB["<START>"] = n
VOCAB["<END>"] = n + 1
DICT = list(VOCAB.keys())

Huấn luyện hai mô hình ngôn ngữ cho ham và spam

In [ ]:
hamModel = buildBigramModel(hamtext, VOCAB, 0.001)
spamModel = buildBigramModel(spamtext, VOCAB, 0.001)

In [ ]:
print(hamModel[-2])
print(sum(hamModel[-2]))

[8.27247348e-04 2.06760147e-07 2.06760147e-07 ... 2.06760147e-07
 2.06760147e-07 2.06760147e-07]
1.0000000000005103


Tạo sinh 10 câu cho ngữ liệu ham và 10 câu cho ngữ liệu spam từ các mô hình đã tạo được.

In [ ]:

print("----------------- Ham messages -----------------")
for i in range(10):
  result = generate(hamModel, VOCAB, DICT, "<START>", 1, 10)
  print(result)
print("----------------- Spam messages -----------------")
for i in range(10):
  result = generate(spamModel, VOCAB, DICT, "<START>", 1, 10)
  print(result)

----------------- Ham messages -----------------
Cool , I deal 1st anna laden . . <END>
Also comes Callertune 1.20 Say every morning princess ! <END>
Oh ... Either have to ur sad FUCK L-oveable <END>
Haha ... fine too . <END>
Let me know there is record at 10 COLLECT <END>
PISS NOTHING <END>
Dis is fantastic bakra V WRK fishhead ST 0871277810810 <END>
Check blood for that number oh . <END>
That day lmao sexy buns ! Ttyl sunlight . <END>
haha <END>
----------------- Spam messages -----------------
Free entry in a å£250 As a video Sunday <END>
Congratulations - Motorola XXXXX.\ ghodbandar 700 Wn support.providing Popcorn <END>
You can stop texts ba breathe suntec -xx harlem <END>
URGENT ! Text FA Cup sabarish Swtheart manageable gandhipuram <END>
Do you . ppm150 eventually words- salmon Sitting weekly <END>
I love reply with weekly Nurungu AL schools 78 <END>
Please reply or text for del Sat night coveragd <END>
FreeMsg anythingtomorrow 87131 roommates Spiral cast daywith maths retired <

2) Mô hình ngôn ngữ theo RNN:

Chúng ta cài đặt lớp SimpleRNN để xác định kiến trúc RNN sẽ huấn luyện mô hình ngôn ngữ. Hàm encode để chuẩn bị ngữ liệu huấn luyện, trong đó chuyển từ thành dạng vector. Hàm train để huấn luyện.

In [ ]:
class SimpleRNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(SimpleRNN, self).__init__()
    self.hiddenSize = hidden_size
    self.in2hid = nn.Linear(input_size + hidden_size, hidden_size)
    self.hid2out = nn.Linear(hidden_size, output_size)
    self.hid2rec = nn.Linear(hidden_size, hidden_size)

  def forward(self, x, hstate):
    xh = torch.cat((x, hstate), 1)
    h = torch.tanh(self.in2hid(xh))
    out = self.hid2out(h)
    rec = self.hid2rec(h)
    return out, rec

  def initState(self):
    return torch.FloatTensor([[1/self.hiddenSize]*self.hiddenSize])

def encode(sentences, dic=VOCAB):
  samples = []
  vocabSize = len(dic)
  for stmp in sentences:
    stmp = ["<START>"] + nltk.word_tokenize(stmp) + ["<END>"]
    v = []
    u = []
    for i in range(len(stmp)-1):
      vtmp = np.zeros(vocabSize)
      id = dic[stmp[i]]
      vtmp[id] = 1
      v.append(vtmp)
      u.append(dic[stmp[i+1]])
    samples.append((v, u))
  return samples

def train(mdl: SimpleRNN, trainset: list, epochs: int, lr: float, device):
  criterion = nn.CrossEntropyLoss()
  mdl = mdl.to(device)
  mdl.train()
  optimizer = torch.optim.Adam(mdl.parameters(), lr=lr)
  for epoch in range(epochs):


    for isample in range(len(trainset)):
      sent, label = trainset[isample]
      rec = mdl.initState().to(device)
      x = torch.LongTensor([sent[0]]).to(device)
      y = torch.LongTensor([label[0]]).to(device)
      out, rec = mdl(x, rec)
      rec = rec.detach()
      loss = criterion(out, y)

      for i in range(1, len(sent)):
        x = torch.LongTensor([sent[i]]).to(device)
        y = torch.LongTensor([label[i]]).to(device)
        out, rec = mdl(x, rec)
        rec = rec.detach()
        loss += criterion(out, y)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if isample % 100 == 0:
        print(epoch, isample, loss.item())



Tạo ngữ liệu huấn luyện cho RNN từ tập sms ham.

In [ ]:
trainset = encode(hamtext)

Huấn luyện mô hình ngôn ngữ với RNN. Lưu ý sử dụng GPU để tăng tốc độ huấn luyện bằng cách đặt device = "cuda"

In [ ]:
device = "cpu" # "cuda" nếu dùng GPU
model = SimpleRNN(len(VOCAB), 150, len(VOCAB))
train(model, trainset, 20, 0.005, device)

<ipython-input-10-b0d8b0bfe0ae>:46: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  x = torch.LongTensor([sent[0]]).to(device)


0 0 233.7337646484375
0 100 97.12450408935547
0 200 125.85655212402344
0 300 113.56658172607422
0 400 53.4500846862793
0 500 153.93215942382812
0 600 44.74412155151367
0 700 29.445175170898438
0 800 50.5957145690918
0 900 84.38468170166016
0 1000 57.817176818847656
0 1100 264.9882507324219
0 1200 221.09205627441406
0 1300 96.68231201171875
0 1400 125.73907470703125
0 1500 318.54315185546875
0 1600 85.53470611572266
0 1700 149.36363220214844
0 1800 78.3237533569336
0 1900 197.27706909179688
0 2000 53.326904296875
0 2100 138.3109588623047
0 2200 197.18301391601562
0 2300 37.576751708984375
0 2400 141.08436584472656
0 2500 65.75645446777344
0 2600 65.34602355957031
0 2700 97.41478729248047
0 2800 174.03659057617188
0 2900 99.41998291015625
0 3000 140.80300903320312
0 3100 25.68910026550293
0 3200 57.17512893676758
0 3300 53.645809173583984
0 3400 48.01947021484375
0 3500 25.72782325744629
0 3600 63.47426986694336
0 3700 227.08131408691406
0 3800 69.35462951660156
0 3900 57.79724884033203


Cài đặt hàm tạo sinh câu từ mô hình RNN đã được huấn luyện.

In [ ]:
def convert(words, dic):
  samples = []
  vocabSize = len(dic)
  for stmp in words:
    vtmp = np.zeros(vocabSize)
    id = dic[stmp]
    vtmp[id] = 1
    samples.append(vtmp)

  return samples
def selectID(arr):
  sel = random.random()
  ex = np.exp(arr)
  prob = ex/np.sum(ex)
  b = 0
  nwords = len(arr)
  for i in range(nwords):
    b += prob[i]
    prob[i] = b
  b = 0
  id = 0
  for i in range(nwords):
    if sel > b and sel <= prob[i]:
      id = i
      break;
    b = prob[i]
  return id

def generate(mdl: SimpleRNN, device, vocab, dictionary , s, n : int):
  genText = []
  sents = []
  sents = s.split()
  sample = convert(sents, vocab)

  inputSize = len(sample[0])
  h = mdl.initState()
  x = torch.LongTensor([sample[0]])

  out, h = mdl(x.to(device), h.to(device))
  genText.append(sents[0])

  for i in range(1, len(sents)):
    x = [torch.LongTensor(sample[i])]
    out, h = mdl(x.to(device), h.to(device))
    genText.append(sents[i])

  id = selectID(out.detach().cpu().numpy()[0])
  predWord = dictionary[id]
  genText.append(predWord)
  if predWord == "<END>":
    return genText
  predArr = np.zeros(inputSize)
  predArr[id] = 1
  predX = torch.LongTensor([predArr])
  for i in range(len(sents), n):
    out, h = mdl(predX.to(device), h.to(device))
    id = selectID(out.detach().cpu().numpy()[0])
    predWord = dictionary[id]
    genText.append(predWord)
    predArr = np.zeros(inputSize)
    predArr[id] = 1
    predX = torch.LongTensor([predArr])
    if predWord == "<END>":
      return genText

  return genText

Tạo sinh 10 msg thuộc nhóm ham

In [ ]:
for i in range(10):
  print(" ".join(generate(model, device, VOCAB, DICT, "<START>", 20)))

BÀI THỰC HÀNH:

Dựa vào các ví dụ, hãy hoàn thành nội dung thực hành của buổi hôm nay, gồm các điểm chính:

- Huấn luyện mô hình ngôn ngữ cho ham và spam, dùng mô hình N-GRAM và RNN nào tốt nhất.

- Tạo sinh 3000 sms cho tập ham và 3000 sms cho tập spam theo mỗi mô hình ngôn ngữ (N-GRAM và RNN)

- Dùng mô hình phân lớp sms-spam tốt nhất đã huấn luyện được, để đánh giá trên hai bộ ngữ liệu đã tạo sinh ở bước trên.